In [11]:
# Importar librerías necesarias
import sys
import os
from datetime import date, timedelta
import json

import os

os.chdir("../Backend")
print("Directorio:", os.getcwd())



Directorio: /home/karen/Documentos/Septimo/AprendizajeAutomatico/GuiasPracticas/proyectoAprendizaje/ProyectoAprendizaje/Backend


In [12]:
# Importar módulos del proyecto
from model.methods import predict_stock_product_date
from db.models import listar_productos
from db.functions import generate_csv
from model.retrain import retrain_from_csv

print(" Módulos importados correctamente")

 Módulos importados correctamente


## 1. Test: /predict/product-date

Predice el stock de un producto específico en una fecha específica.

In [13]:
# Test inicial: Predicción de producto en fecha específica
print("=" * 60)
print("TEST: Predicción de producto en fecha específica")
print("=" * 60)

# Obtener un producto de ejemplo
productos = listar_productos()
producto_ejemplo = productos[0] if productos else "PROD-001"
fecha_ejemplo = (date.today() + timedelta(days=7)).strftime("%Y-%m-%d")

print(f"\nProducto: {producto_ejemplo}")
print(f"Fecha: {fecha_ejemplo}")

resultado = predict_stock_product_date(
    product_id=producto_ejemplo,
    date=fecha_ejemplo
)

print(f"\n Resultado:")
print(json.dumps(resultado, indent=2, default=str))

TEST: Predicción de producto en fecha específica

=== Total de productos: 20 ===

Producto: PROD-001
Fecha: 2025-12-17


2025-12-10 15:31:57.792847: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2025-12-10 15:31:57.793254: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:160] env: CUDA_VISIBLE_DEVICES="-1"
2025-12-10 15:31:57.793264: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:163] CUDA_VISIBLE_DEVICES is set to -1 - this hides all GPUs from CUDA
2025-12-10 15:31:57.793271: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:171] verbose logging is disabled. Rerun with verbose logging (usually --v=1 or --vmodule=cuda_diagnostics=1) to get more diagnostic output from this module
2025-12-10 15:31:57.793277: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:176] retrieving CUDA diagnostic information for host: karen-22
2025-12-10 15:31:57.793282: I external/local_xla/xla/stream_executor/cuda/cuda


 Resultado:
{
  "product_name": "PROD-001",
  "predicted_stock": 36.85,
  "current_stock": 9.0,
  "dias_predichos": 45,
  "predicciones_generadas": 45,
  "predicciones_desde_cache": 0,
  "fecha_inicial": "2025-11-02",
  "fecha_objetivo": "2025-12-17",
  "tipo": "prediccion_recursiva"
}


## 2. Test: /predict/product

Predice el stock de un producto hasta que se agote (máximo 30 días).

In [14]:
# Test inicial: Predicción de producto hasta que se agote
print("=" * 60)
print("TEST: Predicción de producto hasta agotamiento")
print("=" * 60)

producto_ejemplo = productos[0] if productos else "PROD-001"
print(f"\nProducto: {producto_ejemplo}")

day = date.today()
result = []

for i in range(30):
    day = day + timedelta(days=1)
    pred = predict_stock_product_date(
        product_id=producto_ejemplo,
        date=day
    )
    
    result.append({
        "product_name": producto_ejemplo,
        "predicted_stock": int(pred["predicted_stock"]),
        "date": day.strftime("%Y-%m-%d")
    })
    
    if pred["predicted_stock"] <= 0:
        print(f"  Producto se agotará el: {day}")
        break

print(f"\n Resultados ({len(result)} días predichos):")
for r in result[:5]:  # Mostrar solo los primeros 5
    print(f"  - {r['date']}: Stock = {r['predicted_stock']}")
if len(result) > 5:
    print(f"  ... ({len(result)-5} días más)")

TEST: Predicción de producto hasta agotamiento

Producto: PROD-001

 Resultados (30 días predichos):
  - 2025-12-11: Stock = 37
  - 2025-12-12: Stock = 37
  - 2025-12-13: Stock = 38
  - 2025-12-14: Stock = 38
  - 2025-12-15: Stock = 37
  ... (25 días más)


## 3. Test: /predict/date

Predice el stock de todos los productos en una fecha específica.

In [15]:
# Test inicial: Predicción de todos los productos en fecha específica
print("=" * 60)
print("TEST: Predicción de todos los productos en fecha específica")
print("=" * 60)

fecha_ejemplo = (date.today() + timedelta(days=15)).strftime("%Y-%m-%d")
print(f"\nFecha: {fecha_ejemplo}")
print(f"Total de productos: {len(productos)}")

results = []
for product in productos:
    predi = predict_stock_product_date(
        product_id=product,
        date=fecha_ejemplo
    )
    results.append({
        "product_name": product,
        "predicted_stock": int(predi["predicted_stock"]),
        "date": fecha_ejemplo
    })

print(f"\n Resultados (mostrando primeros 5):")
for r in results[:5]:
    print(f"  - {r['product_name']}: Stock = {r['predicted_stock']}")
if len(results) > 5:
    print(f"  ... ({len(results)-5} productos más)")

# Productos con bajo stock
bajo_stock = [r for r in results if r['predicted_stock'] <= 10]
print(f"\n  Productos con stock <= 10: {len(bajo_stock)}")

TEST: Predicción de todos los productos en fecha específica

Fecha: 2025-12-25
Total de productos: 20

 Resultados (mostrando primeros 5):
  - PROD-001: Stock = 37
  - PROD-002: Stock = 42
  - PROD-003: Stock = 48
  - PROD-004: Stock = 31
  - PROD-005: Stock = 43
  ... (15 productos más)

  Productos con stock <= 10: 0


## 4. Test: Generar Reporte CSV

Genera un reporte en formato CSV.

In [17]:
# Test: Generar reporte CSV
print("=" * 60)
print("TEST: Generar Reporte CSV")
print("=" * 60)

# Generar reporte del mes actual
mes_actual = date.today().month
print(f"\nGenerando reporte para el mes: {mes_actual}")

try:
    file_path = generate_csv(mes_actual)
    
    if file_path and os.path.exists(file_path):
        print(f" Archivo CSV generado: {file_path}")
        print(f"Tamaño: {os.path.getsize(file_path)} bytes")
        
        # Leer primeras líneas
        with open(file_path, 'r') as f:
            lines = f.readlines()[:5]
            print("\nPrimeras líneas del CSV:")
            for line in lines:
                print(f"  {line.strip()}")
    else:
        print(" No se pudo generar el archivo CSV")
except Exception as e:
    print(f" Error: {str(e)}")

TEST: Generar Reporte CSV

Generando reporte para el mes: 12
✗ Error en generate_csv: 'int' object has no attribute 'split'
 No se pudo generar el archivo CSV


Traceback (most recent call last):
  File "/home/karen/Documentos/Septimo/AprendizajeAutomatico/GuiasPracticas/proyectoAprendizaje/ProyectoAprendizaje/Backend/db/functions.py", line 162, in generate_csv
    año, mes = map(int, month.split("-"))
AttributeError: 'int' object has no attribute 'split'


## 5. Reentrenamiento del Modelo

Reentrena el modelo con nuevos datos desde un archivo CSV.

In [18]:
# Reentrenamiento con CSV
print("=" * 60)
print("REENTRENAMIENTO DEL MODELO")
print("=" * 60)

# Usar el CSV existente de datos
csv_path = "files/data20oct-2nov.csv"

if os.path.exists(csv_path):
    print(f"\nUsando CSV: {csv_path}")
    print(f"Tamaño: {os.path.getsize(csv_path)} bytes")
    
    # Leer contenido del CSV
    with open(csv_path, 'rb') as f:
        csv_content = f.read()
    
    print("\n⏳ Iniciando reentrenamiento...")
    print("Esto puede tomar varios minutos...")
    
    try:
        resultado = retrain_from_csv(
            csv_content=csv_content,
            filename="data20oct-2nov.csv",
            epochs=10,
            batch_size=128,
            umbral_degradacion=0.1
        )
        
        print("\n Resultado del reentrenamiento:")
        print(json.dumps(resultado, indent=2, default=str))
        
        if resultado.get("success"):
            print("\n Modelo reentrenado exitosamente")
        else:
            print("\n  Reentrenamiento completado con advertencias")
    except Exception as e:
        print(f"\n Error durante el reentrenamiento: {str(e)}")
else:
    print(f" No se encontró el archivo CSV en: {csv_path}")

2025-12-10 15:47:15,227 - ================================================================================
2025-12-10 15:47:15,229 - 🔬 REENTRENAMIENTO - Versión 20251210_154715
2025-12-10 15:47:15,229 - ================================================================================
2025-12-10 15:47:15,230 - 
📥 Paso 1: Cargando datos desde PostgreSQL...
/home/karen/Documentos/Septimo/AprendizajeAutomatico/GuiasPracticas/proyectoAprendizaje/ProyectoAprendizaje/Backend/model/db_loader.py:54: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].fillna(0)
2025-12-10 15:47:15,318 -    ✓ Dataset: 9,800 filas
2025-12-10 15:47:15,318 -    ✓ Rango: 2024-07-01 → 2025-11-02
2025-12-10 15:47:15,319 - 
📦 Paso 2: Cargando modelo actual...


REENTRENAMIENTO DEL MODELO

Usando CSV: files/data20oct-2nov.csv
Tamaño: 81367 bytes

⏳ Iniciando reentrenamiento...
Esto puede tomar varios minutos...


2025-12-10 15:47:15,525 -    🔄 Intentando cargar desde: modelo.h5
2025-12-10 15:47:15,526 -       → Método 1: Carga estándar...
2025-12-10 15:47:15,612 - No training configuration found in the save file, so the model was *not* compiled. Compile it manually.
2025-12-10 15:47:15,613 -       ✓ Cargado con método 1
2025-12-10 15:47:15,614 -    ✓ Modelo cargado exitosamente
2025-12-10 15:47:15,614 -    ✓ Input: (None, 7, 9), Output: (None, 1)
2025-12-10 15:47:15,615 - 
✂️  Paso 3: Preparando datos (split 70/15/15)...
2025-12-10 15:47:15,619 -    ✓ Train: 6,860 filas
2025-12-10 15:47:15,620 -    ✓ Val:   1,470 filas
2025-12-10 15:47:15,620 -    ✓ Test:  1,470 filas
2025-12-10 15:47:15,652 -    ✓ Secuencias: Train=6762, Val=1449, Test=1449
2025-12-10 15:47:15,653 - 
📊 Paso 4: Evaluando modelo actual...
2025-12-10 15:47:16,141 -    📉 RMSE: 0.4206
2025-12-10 15:47:16,142 -    📉 MAE:  0.3803
2025-12-10 15:47:16,143 - 
🎯 Paso 5: Reentrenando modelo (máx 10 épocas)...
2025-12-10 15:47:16,198 -    


 Resultado del reentrenamiento:
{
  "success": true,
  "message": "Reentrenamiento completado. Esperando aprobaci\u00f3n.",
  "training_time_seconds": 7.48,
  "version": "20251210_154715",
  "metricas_anterior": {
    "rmse": 0.4206276659091279,
    "mae": 0.3802847355479976
  },
  "metricas_nuevo": {
    "rmse": 0.21489413166068722,
    "mae": 0.14223379666202152
  },
  "comparacion": {
    "rmse_cambio": 48.91108001747256,
    "mae_cambio": 62.598078921821596
  },
  "recomendacion": {
    "decision": "APROBAR",
    "confianza": "ALTA",
    "razones": [
      "\u2713 RMSE: +48.91%",
      "\u2713 MAE: +62.60%"
    ]
  },
  "datos": {
    "filas_totales": 9800,
    "productos_unicos": 20
  }
}

 Modelo reentrenado exitosamente


## 6. Test Post-Reentrenamiento: /predict/product

Prueba la predicción después del reentrenamiento para comparar resultados.

In [19]:
# Test POST-reentrenamiento: Predicción de producto
print("=" * 60)
print("TEST POST-REENTRENAMIENTO: Predicción de producto")
print("=" * 60)

producto_ejemplo = productos[0] if productos else "PROD-001"
print(f"\nProducto: {producto_ejemplo}")

day = date.today()
result_post = []

for i in range(30):
    day = day + timedelta(days=1)
    pred = predict_stock_product_date(
        product_id=producto_ejemplo,
        date=day
    )
    
    result_post.append({
        "product_name": producto_ejemplo,
        "predicted_stock": int(pred["predicted_stock"]),
        "date": day.strftime("%Y-%m-%d")
    })
    
    if pred["predicted_stock"] <= 0:
        print(f"  Producto se agotará el: {day}")
        break

print(f"\n Resultados POST-reentrenamiento ({len(result_post)} días):")
for r in result_post[:5]:
    print(f"  - {r['date']}: Stock = {r['predicted_stock']}")
if len(result_post) > 5:
    print(f"  ... ({len(result_post)-5} días más)")

print("\n Comparación con resultados anteriores disponible arriba")

TEST POST-REENTRENAMIENTO: Predicción de producto

Producto: PROD-001

 Resultados POST-reentrenamiento (30 días):
  - 2025-12-11: Stock = 37
  - 2025-12-12: Stock = 37
  - 2025-12-13: Stock = 38
  - 2025-12-14: Stock = 38
  - 2025-12-15: Stock = 37
  ... (25 días más)

 Comparación con resultados anteriores disponible arriba


## 7. Test Post-Reentrenamiento: /predict/product-date

Prueba la predicción de producto en fecha específica después del reentrenamiento.

In [20]:
# Test POST-reentrenamiento: Predicción con fecha específica
print("=" * 60)
print("TEST POST-REENTRENAMIENTO: Predicción producto + fecha")
print("=" * 60)

producto_ejemplo = productos[0] if productos else "PROD-001"
fecha_ejemplo = (date.today() + timedelta(days=7)).strftime("%Y-%m-%d")

print(f"\nProducto: {producto_ejemplo}")
print(f"Fecha: {fecha_ejemplo}")

resultado_post = predict_stock_product_date(
    product_id=producto_ejemplo,
    date=fecha_ejemplo
)

print(f"\n Resultado POST-reentrenamiento:")
print(json.dumps(resultado_post, indent=2, default=str))

TEST POST-REENTRENAMIENTO: Predicción producto + fecha

Producto: PROD-001
Fecha: 2025-12-17

 Resultado POST-reentrenamiento:
{
  "product_name": "PROD-001",
  "predicted_stock": 36.85,
  "current_stock": 9.0,
  "dias_predichos": 45,
  "predicciones_generadas": 45,
  "predicciones_desde_cache": 0,
  "fecha_inicial": "2025-11-02",
  "fecha_objetivo": "2025-12-17",
  "tipo": "prediccion_recursiva"
}


## 8. Test Post-Reentrenamiento: /predict/date

Prueba la predicción de todos los productos en fecha específica después del reentrenamiento.

In [21]:
# Test POST-reentrenamiento: Todos los productos en fecha
print("=" * 60)
print("TEST POST-REENTRENAMIENTO: Todos los productos en fecha")
print("=" * 60)

fecha_ejemplo = (date.today() + timedelta(days=15)).strftime("%Y-%m-%d")
print(f"\nFecha: {fecha_ejemplo}")

results_post = []
for product in productos:
    predi = predict_stock_product_date(
        product_id=product,
        date=fecha_ejemplo
    )
    results_post.append({
        "product_name": product,
        "predicted_stock": int(predi["predicted_stock"]),
        "date": fecha_ejemplo
    })

print(f"\n Resultados POST-reentrenamiento (primeros 5):")
for r in results_post[:5]:
    print(f"  - {r['product_name']}: Stock = {r['predicted_stock']}")

bajo_stock_post = [r for r in results_post if r['predicted_stock'] <= 10]
print(f"\n  Productos con stock <= 10: {len(bajo_stock_post)}")

TEST POST-REENTRENAMIENTO: Todos los productos en fecha

Fecha: 2025-12-25

 Resultados POST-reentrenamiento (primeros 5):
  - PROD-001: Stock = 37
  - PROD-002: Stock = 42
  - PROD-003: Stock = 48
  - PROD-004: Stock = 31
  - PROD-005: Stock = 43

  Productos con stock <= 10: 0


## 9. Resumen Final

Resumen de todos los tests realizados.

In [22]:
# Resumen final
print("=" * 60)
print("RESUMEN DE PRUEBAS COMPLETADAS")
print("=" * 60)

tests_completados = [
    " /predict/product-date (PRE)",
    " /predict/product (PRE)",
    " /predict/date (PRE)",
    " Generar reporte CSV",
    " Reentrenamiento del modelo",
    " /predict/product (POST)",
    " /predict/product-date (POST)",
    " /predict/date (POST)",
]

print("\nTests ejecutados:")
for test in tests_completados:
    print(f"  {test}")

print("\n" + "=" * 60)
print("TODAS LAS PRUEBAS COMPLETADAS")
print("=" * 60)

RESUMEN DE PRUEBAS COMPLETADAS

Tests ejecutados:
   /predict/product-date (PRE)
   /predict/product (PRE)
   /predict/date (PRE)
   Generar reporte CSV
   Reentrenamiento del modelo
   /predict/product (POST)
   /predict/product-date (POST)
   /predict/date (POST)

TODAS LAS PRUEBAS COMPLETADAS
